### 사용자 정의 함수
- - -
- 함수기능 : 모델 학습 진행 함수
- 함수이름 : trainning
- 매개변수 : 함수 구동 시 필요한 재료 => 학습을 위한 재료  
            * 모델인스턴스  
            * 학습데이터셋 : 피쳐와 타겟 (학습 전 Tensor화)  
            * 손실함수인스턴스  
            * 최적화인스턴스  
            * 학습횟수 : EPOCH  
            * 배치크기 : BATCH_SIZE 한번에 돌릴양  
            * 배치개수 : BTCH_CNT  
            * 검증용 데이터셋 : 피쳐와 타겟 (검증전 Tensor화)  

- 함수결과 : 학습 시 에포크당 손실값과 성능지표값 , 검증의 손실값과 성능지표값
- - -

* * *
- 함수기능 : 에포크 단위 모델 학습 진행 함수 
- 함수이름 : epochtrainning
- 매개변수 : 함수 구동 시 필요한 재료 => 학습을 위한 재료
            * 모델 인스턴스  
            * 학습용데이터셋 : 피쳐와 타겟 (학습 전 Tensor화)  
            * 손실함수 인스턴스  
            * 최적화 인스턴스  
            * 배치크기 : BATCH_SIZE  
            * 배치개수 : BATCH_CNT  
- 함수결과 : 손실값과 성능지표값  
* * *

In [3]:
# 모듈로딩
import torch
import torch.nn as nn 
import torch.optim as optim
import torch.nn.functional as F 
from torchmetrics.regression import R2Score
from torchmetrics.classification import F1Score

# 텐서 저장 및 실행 위치 설정
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'


In [6]:
## 에포크 당 학습 진행 후 손길값과 성능지표값 반환 함수
def epochTraining(model, feature, target, lossFunc, optimizer, batch_cnt, batch_size=32, is_class=True):
    # - 에포크에서 배치크기만큼 데이터셋 추출 후 학습 진행
    loss_total , score_total =0
    for batch in range(batch_cnt):
        # 배치크기만큼 데이터셋 추출 및 텐서화
        start = batch*batch_size # [예 batch_size = 32 => 0, 32, 64 ..]
        end = start+batch_size   # [예 batch_size=> 32, 64, 96 ..]

        X_train = torch.FloatTensor(feature[start:end].values).to(DEVICE)
        y_train = torch.FloatTensor(target[start:end].values).to(DEVICE)
        
        # 학습 진행
        pre_y = model(X_train).to(DEVICE)

        # 손실 계산
        loss= lossFunc(pre_y, y_train)
        loss_total += loss  # 로스값 누적 

        # 점수 추출
        score = F1Score()(pre_y, y_train) if is_class else R2Score()(pre_y, y_train) # 만약 is_class 면 
        score_total += score

        # 최적화 : w,b업데이트
        optimizer.zero_grad() #gradient를 0으로 초기화
        loss.backword() #  오차 미분, 비용 함수를 미분하여 gradient 계산
        optimizer.step() #  가중치 업데이트 W와 b를 업데이트
    
    # 에포크당 손실값과 성능지표값 반환
    # 테스트 및 검증 함수 사용
    return loss_total/batch_cnt , score/batch_cnt  # 손실점수, 성능지표값 반환

In [ ]:
## 검증 및 테스트 진행 후 손실값과 성능지표값 반환 함수
def testing(model, feature, target, lossFunc, scoreFunc):
    # - 최적화 기능 비활성화 후 데이터셋 추출 후 학습진행

    with torch.no_grad():
            
        # 배치크기만큼 데이터셋 추출 및 텐서화
        featureTS = torch.FloatTensor(feature.values).to(DEVICE)
        targetTS=torch.FloatTensor(target.values).to(DEVICE)

        # 학습 진행
        pre_y = model(featureTS).to(DEVICE)

        # 손실 계산
        loss= lossFunc(pre_y, targetTS)

        # 점수 추출
        score = scoreFunc(pre_y, targetTS)
        

    # 에포크당 손실값과 성능지표값 반환
    # 테스트 및 검증 함수 사용
    return loss , score  # 손실점수, 성능지표값 반환

In [7]:
EPOCHS=10
TV_LOSS={'TRAIN':[], 'VAL':[]}
TV_SCORE={'TRAIN':[], 'VAL':[]}

In [8]:
# 학습 진행
for epoch in range(EPOCHS):
    # 학습 진행
    train_loss, train_score = epochTraining()
    # 검증 진행
    val_loss, val_score = testing()

    # 에포크당 학습 및 검증 결과 저장
    TV_LOSS['TRAIN'].append(train_loss)
    TV_SCORE['TRAIN'].append(train_score)
    TV_LOSS['VAL'].append(val_loss)
    TV_SCORE['VAL'].append(val_score)

    # 에포크당 학습 및 검증 결과 출력
    print(f'[{epoch}/{EPOCHS}]')
    print(f'[TRAIN] LOSS : {train_loss} SCORE : {train_score}')
    print(f'[VAL] LOSS : {val_loss} SCORE : {val_score}')

TypeError: epochTraining() missing 6 required positional arguments: 'model', 'feature', 'target', 'lossFunc', 'optimizer', and 'batch_cnt'